## Problem Solving As Search

### Search Algorithms for Decision Problems

### Artificial Intelligence 1: Week 09

## This video

Recap:
- problem solving as search,
- Search landscapes
- Generate and Test as a common framework

Search methods maintain ordered lists
- that represent the working memory
- in this module we are looking at 'single member' algorithms  
  where we generate test one new solution at a time

This week:
- Uninformed ‘Blind” search: depth/breadth-first

Next Week:  Adding heuristic measures: A*, best-first, hill-climbing


## Recap
- Learning and problem solving can be seen as a search through a set of possible solutions or states.

- Set of candidate solutions + move operator =>landscape

- Sometimes we have quality measures to help guide  search
  - landscape with extra dimension for quality
  - Ideas of local and global optima
- Sometimes not (decision problems)
  - ‘needle in a haystack’ – nothing to guide search
  - example: finding the unlock code for a 5dial combination lock
- Constructive Search: build up partial solutions, complexity 
- Perturbative Search: all solutions have the same complexity


## Recap: Solution = node in search graph / tree
<img src = "figures/search/solution_as_node.png" style="float:right" width =55%>

Depending on your preferred coding paradigm ( and language)
you could either encode solutions

As a data type e.g. a struct in C

*typedef struct {  
  int numbers[N];  
  int depth;  
  int quality;  
  } solution*
     
Or as a class in OOP e.g. python  
Whether you code the move operators  
as a class method, or externally  
is a matter of preference. 

In [ ]:
# there are two options for making solutions as class
# you can wrap up all the testing and move operators in class methods 
class candidateSolutionv0:
    def init():
        self.variableValues = []
        self.quality = 0
        self.depth=0
        
    def Evaluate(self):
        ## code to take encoded variable Values
        ## transform thme into a solution
        ## and measure how good it is
        #e.g. for combination lock
        if(self.variableValues==[5,4,3,2,1]):
            self.quality=1
    
    def ChangeVariableValues(self,positionsToChange=[],newValues = []):
        ## code to enact move operators by
        ## adding varaibleValues (constructive) or 
        ## changing values in some positions
    
mySoln = candidateSolution()
mySoln.variableValues = [1,2,3,4,5]
mySoln.Evaluate()
if(mySoln.quality ==1):
    print("found the unlock code!")

                                

In [ ]:
## OR keep the class definition more generic
## by moving the evaluate and move operators outside

class candidateSolution:
    def __init__(self):
        self.variableValues = []
        self.quality = 0
        self.depth=0
        
def Evaluate(values=[]):
    ## e.g. combination lockl
    if(values == [5,4,3,2,1]):
        return True
    else:
        return False
    
mySoln = candidateSolution()
mySoln.variableValues= [1,2,3,4,5]
if (Evaluate(mySoln.variableValues)):
    mySoln.quality =  1
print(mySoln.quality)    
# then we can have many move operators without changing the class definition
def Increment(values,variableToChange):
    ...

def Decrement(values,variableToChange):
  ...
def swapValues(values, position,position2):
 ... 
        

## Recap: Properties of Search Algorithms

Ways of generating solutions would ideally be:
- Optimal 
- Complete 
- Efficient 
Can't be all three, so **you** (the designer) have to make  a **design decision** about the best trade-offs  for **your** problem

<div> 
    <div style= "float:left" width=25%><img src="figures/search/optimal.png" width=50%></div>
     <div style= "float:left" width=25%><img src="figures/search/complete.png" width=50%> </div>
    <div  style="float:left" width=25%><img src="figures/search/efficient.png" width=50%></div>
</div>    



## Recap: Search using a Generate-test loop

- A common framework we can use to solve many different problems,
  - by changing the representation and  the test() function
- switching between f different algorithms
  - by changing  how we specify Generate() and UpdateWorkingMemory()

    1.   Set WorkingMemory = Empty
    2.   Initialise (CandidateSolution)
    3.   Test ( CandidateSolution)
    4.   UpdateWorkingMemory()
    5.   While ( goal_not_found AND possibilities_left) DO
    6.         CandidateSolution <- Generate ()
    7.	       Test ( CandidateSolution)
    8.         UpdateWorkingMemory()
    9.   OD
    10.  Return (success or failure as appropriate).  

Often we divide working memory into open list and closed list



## Quiz Questions:
- A point that is locally optimal for one landscape, will still be if you change the move operator? [True: False]
- In which if these situations might optimality be less important than efficiency?
  - Speech recognition software for dictation
  - Fingerprint recognition 
  - Neither
  - Both
-Is Exhaustive Search Optimal, Complete and Efficient (True: False x 3]



## Decision Problems and Uninformed search

- Some problems come with a natural measure of quality

- But sometimes we just have a ‘yes/no’ response:
  - Password cracking
  - ‘can I get from A to B’ without [using tool roads | flying | using motorways]?
  - Finding things like files …
  


## Example:

You have a fox, a chicken and a sack of grain. 

You must cross a river with only one of them at a time. 

- If you leave the fox with the chicken he will eat it; 

- If you leave the chicken with the grain he will eat it. 

Can you get all three across safely in less than ten moves?

## What type of problem is this?

“our” bit of the world  is dictated by the rules:
they form the model of our system and the constraints

We are given “goal state” (final output to reach)

So this is an optimisation problem;
- Allowed moves defines a graph.
- The current state is defined by the position of the  fox, chicken, grain, and boat:  
  either  on first bank (0) or second bank (1)
- Seeking sequence of inputs that moves through graph from (0,0,0,0) to (1,1,1,1) 

**Constraints**: fox and chicken,  or chicken and grain can't be on same side without boat
 - i.e. solutions are **infeasible** if they pass through:
   -  {0,0,0,1},{1,1,1,0}   (both problem pairs left unattended)
   -  {0 0, 1,1}, {1,1,0,0}   (fox and chicken unattended)
   -  {0,1,1,0}, {1,0,0,1}  )chicken and grain unattended)


## Diagram of partial graph for this problem
Figure show partial graph for this problem, not all moves below final row shown. 
<img src = "figures/search/fox-chicken-grain-partial-graph.png" width=50%>

## How would you solve this?<img src = "figures/search/fox-chicken-grain-partial-graph.png" style = "float:right" width=30%>

Informally, if you give this to people as an exercise, what they do is:   
- start at one node of graph,
- follow one path e.g. {chicken,boat}->,  <-boat, ...  
  until they reach a problem (INFEASIBLE)   
  (either fox and chicken   
  or chicken and grain on the same side),
- then backtrack to previous “ok” node and try alternative move.

This is an example of Constructive Depth-First Search.




## Common initialisation

    Variables workingCandidate, openList, closedList
    
    ## make initial guess,  test it, then start the openList ##    
    SET workingCandidate = StartSolution
    Evaluate (workingCandidate)
    IF( IsAtGoal(workingCandidate)) ##lucky guess!
        OUTPUT (SUCCESS, workingCandidate)
    APPEND workingCandidate to openList
 

## Depth- First Search Pseudocode   (main loop)
    WHILE ( Openlist not empty) DO       ##main search loop ##
        MOVE (last item from openList into working candidate)
        FOREACH (1-step neighbour)
            neighbour = ApplyMoveOperator(workingCandidate) ## Generate
            Evaluate(neighbor)                              ## Test
	        IF(IsAtGoal(neighbour))
                OUTPUT (SUCCESS, neighbour)
            ELSE IF (neighbor is feasible)                  ## Update Memory
                APPEND( neighbor to end of openList)
            ELSE
                APPEND( neighbor to end of closedList) 
        COPY (working candidate to closedList)
 
    OUTPUT (FAILURE, workingCandidate)     ## if no more solutions to test
 

## What does this look like for fox-chicken-grain problem?

A solution is a sequence of moves of boat with different passengers

There are 8 moves in total {nothing,fox,chicken,grain} X {bank1to2, bank2to1}
- number these from 0 to 7
- candidateSolution.variableValues is a list of moves

**Evaluate()**: 
score is -1 (infeasible), 0 (ok but doesn't reach goal) or 1 (reaches goal)
- starts from state(0,0,0,0)
- apply move referenced in variableValues[0] to get next state
  - if move can't be applied do nothing and leave state unchanged
  - else if next state in forbidden list return INFEASIBLE (-1)
  - else if next state = (1,1,1,1) return SUCCESS (1)
  - else get and apply next move

Choices for ApplyMoveOperator() on Foreach(1-step neighbour) loop;
- perturbative (use *fixed number of d* moves):  
  nested loop through each position (1...n) and value (0...7) changing  a specific move to the new value
  - i.e. each solution has *d* moves and 7d neighbours (7 different values in d different position)  
  
- constructive:  loop through each possible extra move adding that to the *d* existing ones at depth *d*  
  - i.e.  each solution with *d* moves has  8 neighbours, all with *d+1* moves

In [ ]:
from fox_chicken_grain import candidateSolution, Evaluate,TranslateSolutionAsString, IsAtGoal
## Common Initialisation

#Variables workingCandidate, openList, closedList 
workingCandidate = candidateSolution()
openList = []
closedList = []
reason = ""
## make initial guess,  test it, then start the openList ##
## in this case we start with no moves, depth 0, 
## this does nothing so is not at goal but is feasible
workingCandidate.quality=0
atGoal = False
openList.append(workingCandidate)

In [ ]:
iteration=1
import copy
while( atGoal==False and  len(openList)>0 and iteration<50): #WHILE ( Openlist not empty) DO
    print("Iteration {} there are {} candidates on the openList".format(iteration,len(openList)))
    iteration = iteration + 1
    nextItem = len(openList) -1 #MOVE (last item from openList into working candidate)
    workingCandidate = openList.pop(nextItem)
    
    for move in range (8):  #FOREACH (1-step neighbour)-constructive        
        ## Generate ##
        neighbour = copy.deepcopy(workingCandidate)         ## need to make a deep copy so we can change it 
        neighbour.variableValues.append(move)       #neighbour = ApplyMoveOperator(workingCandidate)
        
        ## Test ## 
        Evaluate(neighbour)
        moveList =TranslateSolutionAsString(neighbour)
        if(IsAtGoal(neighbour)):             #IF AT GOAL OUTPUT (SUCCESS, neighbour)
            print('goal found with moves ' +moveList)
            atGoal=True
            break ##takes us out of for loop
            
         ## update Working Memory ##
        elif neighbour.quality==0: #ELSE IF (neighbor is feasible)
            print('  **adding partial solution: '+moveList)
            openList.append(neighbour) 
        else:
            print('    discarding invalid solution: ' +moveList +" because "+reason)
            closedList.append(neighbour)
 
    ##COPY (working candidate to closedList)
    closedList.append(workingCandidate)

if(atGoal==False):##OUTPUT (FAILURE, workingCandidate)
    print('failed to find solution to the problem in the time allowed!')

## Depth-First Search examples

**Constructive**: when
1. you don't know how complex the solution has to be  e.g., fox-chicken-grain type puzzles, tic-tac-toe, or
2. the constraints mean you can test and rule out unproductive branches before you get a complete solution e.g. NQueens

Potential large solutions meansthat you sometimes require *problem-specific code to detect loops* (see problem above)
 

**Perturbative**:    when 
1. you know the complexity and  can only test complete solutions e.g. combination locking cracking, 
2. you can limit the complexity i.e. problem above with only ten moves and 'do-nothing' added as a possible move

Really common idea is to think of the “atomic” move operator  
   i.e. the one that makes the smallest change
 

## Depth-first Perturbative Example: Cracking the code of a combination lock: 
<img src="figures/search/depthfirst-lock.png" style="float:right">
          
          
- atomic move operator is to just change one digit
- construct your tree so that it just makes changes to one digit / tumbler/ wheel  at a time

- changes to the rightmost digit are made first => this is the same as counting!



(   **you'll do this in the lab session**)


## Depth-First Search Characteristics
Efficient:
- Can find solutions quickly.
- Only needs a small amount of storage space:
  - current solution, best seen, plus path followed. 

But not Optimal or Complete:
 - could get stuck for a long time searching an infinite or very deep branch of the tree,
 - especially if recursion is possible.
 - Hard to avoid this for constructive search.  
   - would have to write **problem-specific**  code that tracked what states are visited and flagged loops as infeasible
 -  If using a ‘perturbative’ approach can check whether solution has already been seen before adding it to open list

Implemented as a “stack” system: Last In First Out (LIFO)


## Breadth-First search<img src = "figures/search/fox-chicken-grain-partial-graph.png" style = "float:right" width=30%>
### Basic Idea
Examine all the possible options at each depth/level of the graph  
**before** proceeding to next level down the graph

In the context of **constructive** search this means:  
Examine all the solutionms of a given complexity 
**before** increasing the complexity


## Breadth-First Search Pseudocode (main loop)

    WHILE ( Openlist not empty) DO
      MOVE (first item from openList into working candidate)
      FOREACH (1-step neighbour)
        neighbour = ApplyMoveOperator(workingCandidate) ## Generate ##
        Evaluate(neighbor)             ## Test ## 
	    IF(IsAtGoal(neighbour))
           OUTPUT (SUCCESS, neighbour)
        ELSE IF (neighbor is feasible) ## update Working Memory ##
            APPEND( neighbor to end of openList)
        ELSE
            APPEND( neighbor to end of closedList)
        COPY (working candidate to closedList)
    OUTPUT (FAILURE, workingCandidate) ## if no more solutions to try

 
    
<div class="alert alert-block alert-danger">
    The only difference is the line: <b>    MOVE (first item from openList into working candidate) </b>
    instead of <b>    MOVE (last item from openList into working candidate)</b></div>
    

## Characteristics of Breadth-First Search 

Complete: Guaranteed to find solution if one exists.

Optimal: guaranteed to find closest solution to start

Efficient?
 - Works well when solution is near root,  
   especially if some branches are very deep. 
 - Higher Storage Overheads:  
   especially if branching factor at each node is high,  
   e.g. chess ….
   - have to store each node at current level.
 - have to store current tree – lots of retracing steps.

Implement as a Queue first-in-first-out (FIFO)

Often called “Flood-fill” in games/path-finding 
(because they like to think they’ve invented something)


In [ ]:
#CODE
iteration=1
workingCandidate = candidateSolution()
openList=[]
openList.append(workingCandidate)
atGoal=False

## main breadth-first loop



## Breadth-First perturbative example: combination lock 

The first 10 combinations examined are the same as depth first, but 11th is not.
<img src="figures/search/breadthfirst-lock.png">

# What's in the open list: Depth first 

## Example simple decision problem: find solution with cost <5
Depth First ignores quality. 

Picking the last item is equivalent to sorting the list deepest (youngest) first

<img src="figures/search/depth-with-list.png" width = 50%>

# Same example: breadth-first

Ignores quality and by picking first effectively sorts the list shallowest (oldest) first
<img src="figures/search/breadth-with-list.png" width = 50%>



## Breadth-First vs Depth-First

- Depth-first is often quicker:
  - but may waste time in deep unproductive branches.
  - could apply a depth limit,  
    but then may never find solution.
- Depth-first will return first solution found
   – which may may not be the best.
- Breadth-first often slower, takes more storage, but is
  - “complete” i.e. guaranteed to find solution if one exists,
  - “optimal” i.e. will find the best solution at any given depth.
- Both are “tentative – they allow backtracking.
- Both can be applied to either constructive or perturbative search


## Quiz Questions
- Theseus in the maze with his ball of string, seeking the Minotaur, was doing?
- A search party fanning out and recruiting more people as they consider bigger areas is doing a parallel version of?

- which is ewhich? black numberrs show orde nodesa re examined, whiye numbers show the quality of that node


two figures
depth-tree.png breadth-tree.png

Which is which?
- X is often quicker
   - but may waste time in unproductive branches.
- X will return first solution found
    – that may not be the best / simplest.
    
- Y is often slower, 
- Y takes more storage, 
- but Y  is
  - “complete” i.e. guaranteed to find solution if one exists,
  - “optimal” i.e. will find the best solution at any given depth.



## Summary

Decision problems:    
- only a 'yes/no' answer
- can have multiple solutions with different complexity
- often associated with **Constraint Satisfaction Problems**

**Breadth-first** and **Depth-first**  are 'blind' or 'uninformed' search algorithms.  

You need to understand and be able to recognise:
 - when to apply them
 - what their characteristics are
 
If we give you a scenario you shouilfd be able ot selerct an appropriuae method and justyifyu your choice.

### Next week:    search algorirthms for problem-solving guided by  a quality/cost function 
 